In [5]:
import boto3
import tarfile
import os

# Configure your AWS credentials (replace with your own credentials)
aws_access_key_id = ''
aws_secret_access_key = ''
region_name = 'eu-north-1'
bucket_name = 'indian-accent-english'

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Define the S3 URI to the TAR file
s3_uri = f"s3://{bucket_name}/svarah.tar"

# Specify the local file path where you want to save the downloaded file
local_file_path = './svarah_downloaded.tar'

# Extract the bucket name and object key from the S3 URI
bucket_name = s3_uri.split('//')[1].split('/')[0]
object_key = '/'.join(s3_uri.split('//')[1].split('/')[1:])

# Download the TAR file from S3
s3.download_file(bucket_name, object_key, local_file_path)

# Verify that the file has been downloaded
if os.path.exists(local_file_path):
    print(f"File downloaded to {local_file_path}")
else:
    print("Download failed.")


File downloaded to ./svarah_downloaded.tar


In [6]:
import shutil

In [7]:
shutil.unpack_archive("./svarah_downloaded.tar", "./extracted_data")

In [1]:
!pip install librosa soundfile torchaudio datasets transformers accelerate evaluate tensorboardX jiwer

  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.2 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/31/52/3f76cf4364322db27ea1d6b32a7864b2d7a78a2c240d975b25783e262035/torchaudio-2.1.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/7c/55/b3432f43d6d7fee999bb23a547820d74c48ec540f5f7842e41aa5d8d5f3a/datasets-2.14.6-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/9a/06/e4ec2a321e57c03b7e9345d709d554a52c33760e5015fdff0919d9459af0/transformers-4.35.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 21.9 MB/s eta 0:00

In [2]:
import os

# Define the directory containing your text files
text_files_directory = './extracted_data/svarah/audio/'

# Define the output file paths for scp_entries and txt_entries
scp_entries_file = './extracted_data/scp_entries.txt'
txt_entries_file = './extracted_data/txt_entries.txt'

# Initialize lists to store entries
scp_entries = []
txt_entries = []

# Set the limit for the number of instances you want to add
limit_instances = 10

# Iterate through text files in the directory
for text_file_name in os.listdir(text_files_directory):
    if text_file_name.endswith('.txt'):
        text_file_path = os.path.join(text_files_directory, text_file_name)
        with open(text_file_path, 'r') as text_content:
            for line in text_content:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    audio_path, audio_id, transcription = parts
                    audio_path = os.path.join(audio_path)  # You may need to adjust this based on your folder structure

                    # Create an entry for scp_entries
                    scp_entry = f"{audio_id} ./extracted_data/svarah/{audio_path}\n"
                    scp_entries.append(scp_entry)

                    # Create an entry for txt_entries
                    txt_entry = f"{audio_id} {transcription}\n"
                    txt_entries.append(txt_entry)

                if len(scp_entries) >= limit_instances:
                    # If the limit is reached, break out of the loop
                    break

    if len(scp_entries) >= limit_instances:
        # If the limit is reached, break out of the loop
        break

# Write the scp_entries and txt_entries to their respective files
with open(scp_entries_file, 'w') as scp_file:
    scp_file.writelines(scp_entries)

with open(txt_entries_file, 'w') as txt_file:
    txt_file.writelines(txt_entries)

print(f'SCP entries have been written to {scp_entries_file}')
print(f'Text entries have been written to {txt_entries_file}')


SCP entries have been written to ./extracted_data/scp_entries.txt
Text entries have been written to ./extracted_data/txt_entries.txt


In [3]:
from datasets import Dataset, Audio, Value

In [4]:
scp_entries = open('./extracted_data/scp_entries.txt', 'r').readlines()
txt_entries = open("./extracted_data/txt_entries.txt", 'r').readlines()

if len(scp_entries) == len(txt_entries):
    audio_dataset = Dataset.from_dict({"audio": [audio_path.split()[1].strip() for audio_path in scp_entries],
                    "sentence": [' '.join(text_line.split()[1:]).strip() for text_line in txt_entries]})

    audio_dataset = audio_dataset.cast_column("audio", Audio(sampling_rate=16000))
    audio_dataset = audio_dataset.cast_column("sentence", Value("string"))
    audio_dataset.save_to_disk('./extracted_data/svarah/audio')
    print(audio_dataset)
    print('Data preparation done')

else:
    print('Please re-check the audio_paths and text files. They seem to have a mismatch in terms of the number of entries. Both these files should be carrying the same number of lines.')

Casting the dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 10
})
Data preparation done


In [5]:
audio_dataset

Dataset({
    features: ['audio', 'sentence'],
    num_rows: 10
})

In [6]:
import torch
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from datasets import DatasetDict, Audio, load_from_disk, concatenate_datasets
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [7]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/audio_utils.py:194: RuntimeWarning: divide by zero encountered in true_divide
  enorm = 2.0 / (filter_freqs[2 : num_mel_filters + 2] - filter_freqs[:num_mel_filters])


In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [9]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [10]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [11]:
audio_dataset = audio_dataset.map(prepare_dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [14]:
import evaluate

metric = evaluate.load("wer")

In [15]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [16]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [17]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-indian-accent",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=1,
    max_steps=2,
    gradient_checkpointing=True,
    fp16=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1,
    eval_steps=1,
    logging_steps=1,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [20]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=audio_dataset,
    eval_dataset=audio_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss
